# Practical session 9: collecting data **solutions**

Wednesday 6 December 2023

In this practical session, we'll explore how to collect data in order to create our own datasets. We'll explore data collection in two different ways. First we'll look at data collection through APIs, i.e. using a programming interface which allows us to query for data in a structured way. Secondly, we'll look at web scraping, where we'll extract the data we're after from - sometimes quite messy - web pages that are meant for human reading, rather than automatic  processing.

## API

**API** is an acronym for Application Programming Interface. It is the general term for an interface that allows two computer programs to communicate with each other. An API can therefore be accessed programmatically: it consists of methods to receive requests and send responses. We'll use the term API to designate a **web** API. Developers often make use of web APIs to build applications using someone else's content; and we can equally query the API in order to construct a dataset based on that content.

Why do web APIs exist? They allow us to access data in a structured way, using structured requests. The same information is usually also available on the web, in a standard webpage format. However, these webpages are designed to be read by humans - not by machines - and it will be much more difficult to extract the information we're after (extracting information from standard web pages is called **web scraping**, and it will be the second method we explore in the latter part of this notebook).

First, let's import a couple of modules and functions that we'll need. The `requests` module allows us to download data from the internet, using the HTTP protocol. This is the common way to access APIs. We'll also import the `json` module, which provides us with a json encoder/decoder (APIs often return data in this structured format). And the `pprint` function, which stands for *pretty print*, is a function that allows us to print our data structures in a cleaner, more readable way.

In [ ]:
import requests
import json
from pprint import pprint

### A simple API call

Let's reconsider the example we looked at during the lecture: we'll make use of the [Open Notify](http://open-notify.org/) API. Specifically, we'll make use of the `astros.json` endpoint, which will provide us with an overview of the people that are currently in space. We'll make use of a GET request in order to retrieve the data; this is carried out using the `get` method of the `requests` module:

In [ ]:
response = requests.get("http://api.open-notify.org/astros.json")

The answer to our request is put in a `Response` object; we can see the object has code 200, which is the HTTP status code for success, and thus indicates that we have successfully acquired the requested data.

In [ ]:
response

We can now access the data that has been stored in the response object; the headers contain technical metadata:

In [ ]:
response.headers

But we are interested in the actual response we got back from the server, which is encoded as a structured JSON string:

In [ ]:
response.text

### Convert JSON string to Python object

During the lecture, we explored the JSON format a bit further. We can decode the format to a nested Python data structure using the `loads` (load string) method of the `json` module:

In [ ]:
astro_data  = json.loads(response.text)

Our `astro_data` is now a Python object. We can use the function `pprint` to print it out in a readable way (compare with a standard `print`):

In [ ]:
pprint(astro_data)

We can see our data structure consists of a dictionary with three keys: *message*, *number*, and *people*. The value of the key *people* is itself a list, which consists of all the persons in space; and each person is modeled as a dictionary, which contains the keys *craft* (the value of this key represents the spacecraft the person is located in) and *name* (the value of this key is the name of the actual astronaut).

We can also use the `json` method of the response object; this will equally parse the json string, and turn it into a Python object (under the hood, the parser from the `json` module is used):

In [ ]:
astro_data2 = response.json()

In [ ]:
pprint(astro_data2)

In [ ]:
astro_data == astro_data2

As we now have a Python object, we can make use of indexing and for-loops:

In [ ]:
for person in astro_data['people']:
    print(person['name'])

#### Exercise 1

The Open Notify API equally has an an endpoint that returns the current position (in latitude and longitude) of the ISS space station. You can find the [API documentation over here](http://open-notify.org/Open-Notify-API/ISS-Location-Now/). First, make sure the the modules `geopandas` and `shapely` are installed:

In [ ]:
!pip install geopandas
!pip install shapely

Next, send a request to the API in order to receive the current latitude and longitude values, and make sure these two values are assigned to two separate Python variables named `latitude` and `longitude`. Make sure the two variables are of the correct type.

In [ ]:
# INSERT YOUR CODE HERE

response = requests.get("http://api.open-notify.org/iss-now.json")
iss_data = response.json()
latitude = float(iss_data['iss_position']['latitude'])
longitude = float(iss_data['iss_position']['longitude'])

When your code is finished, you can execute the code cell below; if your code is correct, you will get a map of the world, with a little dot in red providing the current location of the ISS.

In [ ]:
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

geometry = [Point((longitude, latitude))]
gdf = GeoDataFrame(geometry=geometry)

#the next command loads a simple map provided by geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

gdf.plot(ax=world.plot(figsize=(10, 6)),
         marker='o', color='red', markersize=15);

### API call with query parameters

The calls above are fairly simple: there is just a single block of data offered by the APIs. Often, APIs will take query parameters, which can be used to query for specific data. As an example, we'll use the [nationalize.io](https://nationalize.io/) API.

The API takes a single `name` parameter; we can define this as a dictionary with one key-value pair, and provide this dictionary as an argument to the `params` parameter of the `get` method:

In [ ]:
query = {'name' : 'Charlotte'}

response = requests.get("https://api.nationalize.io", params=query)

What the `get` method is doing under the hood, is generating a URL that will be used to query the API:

In [ ]:
response.url

Let's inspect the result:

In [ ]:
pprint(response.json())

If you consult the [API documentation](https://nationalize.io/), you'll see that you can also provide a list of names, as below (note that we are adding empty square brackets to the `name` key to indicate to the API we are providing a list):

In [ ]:
query = {'name[]' : ['Robbe', 'Liesbeth']}

response = requests.get("https://api.nationalize.io", params=query)

In [ ]:
pprint(response.json())

### iRail API

Finally, we'll look at the [iRail API](https://docs.irail.be/), which makes available the railway time schedules in Belgium. This API provides different endpoints, which you can find in the documentation. As an example, the following API request asks for the next couple of trains (6 by default) that leave from Leuven station to Brussels Central station, on 6 December 2023 at 8:30am.

In [ ]:
query = {'format' : 'json',
         'lang': 'en',
         'from': 'leuven',
         'to': 'brussels-central',
         'date': '110124',
         'time': '0830'
}

response = requests.get("https://api.irail.be/connections/", params=query)
results = response.json()

Explore the results you get back (i.e. the `results` variable).


In [ ]:
results

Note: the times that are returned by the API are in so called [unix time](https://en.wikipedia.org/wiki/Unix_time). In order to convert them, we'll make use of the `datetime` module (note that we'll convert the time to universal time (UTC); Central European Time is one hour ahead of UTC).

In [ ]:
import time
from datetime import datetime

We can now display the departure time, arrival time, and train direction for each connection like this:

In [ ]:
for connection in results['connection']:
    dep_time = int(connection['departure']['time'])
    arr_time = int(connection['arrival']['time'])
    direction = connection['arrival']['direction']['name']

    print('Leaving at {dep}, arriving at {arr}, direction {dir}'.format(
        dep=datetime.utcfromtimestamp(dep_time),
        arr=datetime.utcfromtimestamp(arr_time),
        dir=direction)
    )

#### Exercise 2

Using the iRail API, display all trains that leave from Leuven station within the next half hour from the current time. Note: you can get the current time (in unix time) using the command `time.time()`.

In [ ]:
# INSERT YOUR CODE HERE

query = {'format' : 'json',
         'lang': 'en',
         'station': 'leuven',
}

response = requests.get("https://api.irail.be/liveboard/", params=query)
results = response.json()

# half an hour from now is current time + (30 * 60 seconds)
time_limit = time.time() + (30 * 60)

for departure in results['departures']['departure']:
  dep_time = int(departure['time'])
  if dep_time < time_limit:
    destination = departure['station']
    print('{time} to {dest}'.format(time=datetime.utcfromtimestamp(dep_time),
                                    dest=destination))

## Web scraping

If there is no API available for the data we need, we can always try to directly scrape them from the web. For this, we can use the module `BeautifulSoup`.

In [ ]:
from bs4 import BeautifulSoup
import requests

We can use `BeautifulSoup` to quickly extract the plain text from a web page. To do so, we'll download an HTML page from the web (using the `requests` module, and then parse its content with `BeautifulSoup`.

In [ ]:
req = requests.get('https://www.theguardian.com')
soup = BeautifulSoup(req.text, 'html.parser')

 We can then use the method `get_text` to extract the plain text from the page.

In [ ]:
print(soup.get_text())

Web pages are structured using [HTML markup](https://en.wikipedia.org/wiki/HTML#Markup). Often, we can make use of the tags of the web page in order to extract the specific data we're after. In the following example, we'll download [a page that contains 50 common proverbs in English](https://www.engvid.com/english-resource/50-common-proverbs-sayings/), and we'll make use of the HTML table tags (`td`), together with specific class names, in order to extract the data we're after. To explore the HTML source of a web page, you browser will usually have an option that says something like `Page Source`. Use this option to locate the proverbs, definitions, and examples we want to extract within the source of the page. Then, execute the code below.

In [ ]:
req = requests.get('https://www.engvid.com/english-resource/50-common-proverbs-sayings/')
soup = BeautifulSoup(req.text, 'html.parser')

proverbs = soup.find_all('td', {'class': 'res_proverb_proverb'})
definitions = soup.find_all('td', {'class': 'res_proverb_def'})
examples = soup.find_all('td', {'class': 'res_proverb_example'})

Using this code, we extract the proverbs, definitions, and examples in three separate lists, based on the `td` tag and the specific class names that are used to encode the three entities within the webpage. We can now combine that data into one big list (where each element contains a proverb, its definition, and an example). For this, we'll use the [zip](https://docs.python.org/3/library/functions.html#zip) function, which "zips" together different lists:

In [ ]:
all_proverbs = []
for p, d, e in zip(proverbs, definitions, examples):
    all_proverbs.append([p.text, d.text, e.text])

It is now also possible to import this data into Pandas:

In [ ]:
import pandas as pd

pd.DataFrame(all_proverbs, columns=['proverb', 'definition', 'example'])

Of course, BeautifulSoup has much more functionality, which you can read about in [the documentation](https://beautiful-soup-4.readthedocs.io/en/latest/).

#### Exercise 3

The webpage [https://www.loc.gov/rr/print/list/057_chron.html](https://www.loc.gov/rr/print/list/057_chron.html) contains a table of US presidents in chronological order, together with the years in office, the first lady, and the vice-president. Extract this information from the page, and put it in a structured Pandas dataframe.

In [ ]:
# INSERT YOUR CODE HERE

req = requests.get('https://www.loc.gov/rr/print/list/057_chron.html')
soup = BeautifulSoup(req.text, 'html.parser')

extracted_data = []

# we can extract the table we're after based on the border
table = soup.find('table', {'border': '1'})

# from this table, we select all rows
all_rows = table.find_all('tr')

# for each row, we select the four cells
for row in all_rows:
  all_cells = row.find_all('td')
  if all_cells:
    (dates, president, first_lady, vice_president) = all_cells
    extracted_data.append([dates.text, president.text,
                           first_lady.text, vice_president.text])

pd.DataFrame(extracted_data, columns=['dates', 'president',
                                      'first lady', 'vice-president'])